處理 benckmarkdataset 的資料

In [19]:
%matplotlib inline
import numpy as np
import pandas as pd
import csv
from sklearn.svm import SVC
from itertools import product
from sklearn.metrics import accuracy_score, confusion_matrix

with open('benchmarkdataset_train.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
bench_train = pd.DataFrame(data)

print(bench_train)
print()

#--------------------------------------------------------------
# 計算每種胺基酸的出現次數
amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
count_matrix_train = np.zeros((len(sequences), len(amino_acids))) 
percentage_matrix_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_train[i, j] = sequence.count(amino_acid)
    percentage_matrix_train.append(count_matrix_train[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_train = np.array(percentage_matrix_train)

# 顯示比例矩陣
df_percentage_train = pd.DataFrame(percentage_matrix_train, columns=list(amino_acids))
print('AAC:')
print(df_percentage_train)

# 合併特徵資料和目標資料
df_combined_train = pd.concat([df_percentage_train, bench_train['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_train['Target'] = df_combined_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_train, y_train = df_combined_train.drop(['Header', 'Target'], axis=1), df_combined_train['Target']


#--------------------------------------------------------------
# 創建所有二個胺基酸相連的標籤
DPC = [a + b for a in amino_acids for b in amino_acids]

# 計算每對胺基酸的出現次數
count_DPC_train = np.zeros((len(sequences), len(DPC)))
percentage_DPC_train = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_train[i, j] = sequence.count(aa_pair)
    percentage_DPC_train.append(count_DPC_train[i]/len(sequence))

# 計算每對胺基酸在序列中的比例
percentage_DPC_train = np.array(percentage_DPC_train)

# 顯示比例矩陣
df_percentage_DPC_train = pd.DataFrame(percentage_DPC_train, columns=DPC)
print('DPC:')
print(df_percentage_DPC_train)

# 合併特徵資料和目標資料
df_combined_DPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, bench_train['Header']], axis=1)
print('AAC + DPC:')
print(df_combined_DPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_train['Target'] = df_combined_DPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_train, y_DPC_train = df_combined_DPC_train.drop(['Header', 'Target'], axis=1), df_combined_DPC_train['Target']

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_train = np.zeros((len(sequences), len(TPC)))
percentage_TPC_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_train[i, j] = sequence.count(amino_acid_combination)
    percentage_TPC_train.append(count_TPC_train[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_TPC_train = np.array(percentage_TPC_train)

# 顯示比例矩陣
df_percentage_TPC_train = pd.DataFrame(percentage_TPC_train, columns=TPC)
print('TPC:')
print(df_percentage_TPC_train)

# 合併特徵資料和目標資料
df_combined_TPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, df_percentage_TPC_train, bench_train['Header']], axis=1)
print('AAC + DPC + TPC:')
print(df_combined_TPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_train['Target'] = df_combined_TPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_train, y_TPC_train = df_combined_TPC_train.drop(['Header', 'Target'], axis=1), df_combined_TPC_train['Target']

      Header                Sequence
0      >AA29         FLKDHRISTFKNWPF
1      >AA30    FLSSRLQDLYSIVRRADRAA
2      >AA31            GDVIDTDRDIDR
3      >AA32          GFHDHGPCDPPSHK
4      >AA33       GHRATSDLASTGEESQD
..       ...                     ...
208  >neg131       VVRLAREPGKRESRYMH
209  >neg132       YEDLRDESLKGLVDIGF
210  >neg133  YFLIQSVSSTVMLLNGLYIFVN
211  >neg134         YGEPGMQLFVYGREE
212  >neg135    YNLSDTIKAFSILLLTDLCI

[213 rows x 2 columns]

AAC:
            A         C         D         E         F         G         H  \
0    0.000000  0.000000  0.066667  0.000000  0.200000  0.000000  0.066667   
1    0.150000  0.000000  0.100000  0.000000  0.050000  0.000000  0.000000   
2    0.000000  0.000000  0.416667  0.000000  0.000000  0.083333  0.000000   
3    0.000000  0.071429  0.142857  0.000000  0.071429  0.142857  0.214286   
4    0.117647  0.000000  0.117647  0.117647  0.000000  0.117647  0.058824   
..        ...       ...       ...       ...       ...       ... 

In [20]:
with open('benchmarkdataset_test.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
bench_test = pd.DataFrame(data)

amino_acids = 'ACDEFGHIKLMNPQRSTVWY'


#--------------------------------------------------------------
# 計算每種胺基酸的出現次數
count_matrix_test = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_test[i, j] = sequence.count(amino_acid)
    percentage_matrix_test.append(count_matrix_test[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_test = np.array(percentage_matrix_test)

# 顯示比例矩陣
df_percentage_test = pd.DataFrame(percentage_matrix_test, columns=list(amino_acids))
print('AAC:')
print(df_percentage_test)

# 合併特徵資料和目標資料
df_combined_test = pd.concat([df_percentage_test, bench_test['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_test['Target'] = df_combined_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_test, y_test = df_combined_test.drop(['Header', 'Target'], axis=1), df_combined_test['Target']

#--------------------------------------------------------------
# 計算每對胺基酸的出現次數
count_DPC_test = np.zeros((len(sequences), len(DPC)))
percentage_DPC_test = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_test[i, j] = sequence.count(aa_pair)
    percentage_DPC_test.append(count_DPC_test[i]/len(sequence))

# 計算每對胺基酸在序列中的比例
percentage_DPC_test = np.array(percentage_DPC_test)

# 顯示比例矩陣
df_percentage_DPC_test = pd.DataFrame(percentage_DPC_test, columns=DPC)
print('DPC:')
print(df_percentage_DPC_test)

# 合併特徵資料和目標資料
df_combined_DPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, bench_test['Header']], axis=1)
print('Combine AAC, DPC:')
print(df_combined_DPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_test['Target'] = df_combined_DPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_test, y_DPC_test = df_combined_DPC_test.drop(['Header', 'Target'], axis=1), df_combined_DPC_test['Target']

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_test = np.zeros((len(sequences), len(TPC)))
percentage_TPC_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_test[i, j] = sequence.count(amino_acid_combination)
    percentage_TPC_test.append(count_TPC_test[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_TPC_test = np.array(percentage_TPC_test)

# 顯示比例矩陣
df_percentage_TPC_test = pd.DataFrame(percentage_TPC_test, columns=TPC)
print('TPC:')
print(df_percentage_TPC_test)

# 合併特徵資料和目標資料
df_combined_TPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, df_percentage_TPC_test, bench_test['Header']], axis=1)
print('Combine AAC, DPC, TPC:')
print(df_combined_TPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_test['Target'] = df_combined_TPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_test, y_TPC_test = df_combined_TPC_test.drop(['Header', 'Target'], axis=1), df_combined_TPC_test['Target']

AAC:
           A         C         D         E         F         G         H  \
0   0.157895  0.105263  0.000000  0.052632  0.157895  0.105263  0.052632   
1   0.058824  0.000000  0.058824  0.029412  0.029412  0.029412  0.029412   
2   0.350000  0.000000  0.050000  0.050000  0.000000  0.000000  0.000000   
3   0.157895  0.105263  0.000000  0.000000  0.000000  0.157895  0.000000   
4   0.157895  0.105263  0.000000  0.052632  0.105263  0.105263  0.052632   
5   0.000000  0.058824  0.294118  0.000000  0.000000  0.000000  0.000000   
6   0.000000  0.181818  0.090909  0.181818  0.000000  0.000000  0.000000   
7   0.080000  0.060000  0.000000  0.080000  0.040000  0.060000  0.020000   
8   0.074627  0.089552  0.074627  0.059701  0.044776  0.044776  0.014925   
9   0.083333  0.166667  0.000000  0.000000  0.000000  0.083333  0.250000   
10  0.000000  0.000000  0.600000  0.000000  0.000000  0.000000  0.000000   
11  0.100000  0.050000  0.125000  0.025000  0.075000  0.100000  0.000000   
12  0.0

使用 default param 的 RandomForestClassifier(param)

In [27]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

# Single
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')


# DPC
rf_classifier.fit(X_DPC_train, y_DPC_train)
y_DPC_pred = rf_classifier.predict(X_DPC_test)

accuracy_DPC = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy_DPC * 100:.2f}%')


# TPC
rf_classifier.fit(X_TPC_train, y_TPC_train)
y_TPC_pred = rf_classifier.predict(X_TPC_test)

accuracy_TPC = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy_TPC * 100:.2f}%')

Model Accuracy(AAC): 74.55%
Model Accuracy(DPC): 69.09%
Model Accuracy(TPC): 61.82%


先對 X 作 Select From Model，再使用 default param 的 RandomForestClassifier(param) 進行分類

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

rf_classifier = RandomForestClassifier()


# --------------------------------------------------------------
rf_classifier.fit(X_train, y_train)
#feature_importance = rf_classifier.feature_importances_
sfm = SelectFromModel(rf_classifier, threshold=0.01)  # 可以調整閾值
sfm.fit(X_train, y_train)

X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

selected_features = X_train.columns[sfm.get_support()]

# 印出篩選後的特徵
# print("AAC Selected Features:")
# print(selected_features)

rf_classifier.fit(X_train_selected, y_train)

y_pred = rf_classifier.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')



# --------------------------------------------------------------
rf_classifier.fit(X_DPC_train, y_DPC_train)
#feature_importance = rf_classifier.feature_importances_
sfm = SelectFromModel(rf_classifier, threshold=0.01)  # 可以調整閾值
sfm.fit(X_DPC_train, y_DPC_train)

X_DPC_train_selected = sfm.transform(X_DPC_train)
X_DPC_test_selected = sfm.transform(X_DPC_test)

selected_features = X_DPC_train.columns[sfm.get_support()]

# 印出篩選後的特徵
# print("DPC Selected Features:")
# print(selected_features)

rf_classifier.fit(X_DPC_train_selected, y_DPC_train)

y_DPC_pred = rf_classifier.predict(X_DPC_test_selected)
accuracy_DPC = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy_DPC * 100:.2f}%')


# --------------------------------------------------------------
rf_classifier.fit(X_TPC_train, y_TPC_train)
#feature_importance = rf_classifier.feature_importances_
sfm = SelectFromModel(rf_classifier, threshold=0.01)  # 可以調整閾值
sfm.fit(X_TPC_train, y_TPC_train)

X_TPC_train_selected = sfm.transform(X_TPC_train)
X_TPC_test_selected = sfm.transform(X_TPC_test)

selected_features = X_TPC_train.columns[sfm.get_support()]

# 印出篩選後的特徵
# print("TPC Selected Features:")
# print(selected_features)

rf_classifier.fit(X_TPC_train_selected, y_TPC_train)

y_TPC_pred = rf_classifier.predict(X_TPC_test_selected)
accuracy_TPC = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy_TPC * 100:.2f}%')

先用 Grid Search 找出最佳超參數 param， 再對 X 作 Select From Model，最後使用 RandomForestClassifier(param) 進行分類

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
import time

# 定義參數範圍
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf_classifier = RandomForestClassifier()

# -------------------------------------------------
print('-- AAC')
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')

start_time = time.time()
grid_search.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time
print(f"Grid Search Execution Time: {execution_time:.2f} seconds")

# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)

best_model = grid_search.best_estimator_

#feature_importance = rf_classifier.feature_importances_
sfm = SelectFromModel(rf_classifier, threshold=0.01)  # 可以調整閾值
sfm.fit(X_train, y_train)

X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)

selected_features = X_train.columns[sfm.get_support()]

# 印出篩選後的特徵
# print("Selected Features:")
# print(selected_features)

# 將模型擬合到篩選後的訓練數據上
best_model.fit(X_train_selected, y_train)

y_pred = best_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')


# -------------------------------------------------
print('-- DPC')
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')

start_time = time.time()
grid_search.fit(X_DPC_train, y_DPC_train) # fix: grid_search.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time
print(f"Grid Search Execution Time: {execution_time:.2f} seconds")

# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)

best_model = grid_search.best_estimator_

#feature_importance = rf_classifier.feature_importances_
sfm = SelectFromModel(rf_classifier, threshold=0.01)  # 可以調整閾值
sfm.fit(X_DPC_train, y_DPC_train)

X_DPC_train_selected = sfm.transform(X_DPC_train)
X_DPC_test_selected = sfm.transform(X_DPC_test)

selected_features = X_DPC_train.columns[sfm.get_support()]

# 印出篩選後的特徵
# print("DPC Selected Features:")
# print(selected_features)

best_model.fit(X_DPC_train_selected, y_DPC_train)

y_DPC_pred = best_model.predict(X_DPC_test_selected)
accuracy_DPC = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy_DPC * 100:.2f}%')


# -------------------------------------------------
print('-- TPC')
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy')

start_time = time.time()
grid_search.fit(X_TPC_train, y_TPC_train) # fix: grid_search.fit(X_train, y_train)
end_time = time.time()
execution_time = end_time - start_time
print(f"Grid Search Execution Time: {execution_time:.2f} seconds")

# best_params = grid_search.best_params_
# print("Best Parameters:", best_params)

best_model = grid_search.best_estimator_

#feature_importance = rf_classifier.feature_importances_
sfm = SelectFromModel(rf_classifier, threshold=0.01)  # 可以調整閾值
sfm.fit(X_TPC_train, y_TPC_train)

X_TPC_train_selected = sfm.transform(X_TPC_train)
X_TPC_test_selected = sfm.transform(X_TPC_test)

selected_features = X_TPC_train.columns[sfm.get_support()]

# 印出篩選後的特徵
# print("TPC Selected Features:")
# print(selected_features)

best_model.fit(X_TPC_train_selected, y_TPC_train)

y_TPC_pred = best_model.predict(X_TPC_test_selected)
accuracy_TPC = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy_TPC * 100:.2f}%')

先過 StandardScaler，再做 PCA ，最後使用 default 的 RandomForestClassifier(param)

In [78]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# -------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

pca = PCA(n_components=0.6)

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_pca, y_train)

y_pred = rf_classifier.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')


# -------------------------------------------------
scaler = StandardScaler()
X_DPC_train_scaled = scaler.fit_transform(X_DPC_train)
X_DPC_test_scaled = scaler.transform(X_DPC_test)

pca = PCA(n_components=0.6)

X_DPC_train_pca = pca.fit_transform(X_DPC_train_scaled)
X_DPC_test_pca = pca.transform(X_DPC_test_scaled)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_DPC_train_pca, y_DPC_train)

y_DPC_pred = rf_classifier.predict(X_DPC_test_pca)
accuracy = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy * 100:.2f}%')


# -------------------------------------------------
scaler = StandardScaler()
X_TPC_train_scaled = scaler.fit_transform(X_TPC_train)
X_TPC_test_scaled = scaler.transform(X_TPC_test)

pca = PCA(n_components=0.6)

X_TPC_train_pca = pca.fit_transform(X_TPC_train_scaled)
X_TPC_test_pca = pca.transform(X_TPC_test_scaled)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_TPC_train_pca, y_TPC_train)

y_TPC_pred = rf_classifier.predict(X_TPC_test_pca)
accuracy = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy * 100:.2f}%')

Model Accuracy(AAC): 70.91%
Model Accuracy(DPC): 58.18%
Model Accuracy(TPC): 60.00%


先過 StandardScaler，再過 RFE，最後使用 default 的 RandomForestClassifier(param)

In [57]:
from sklearn.feature_selection import RFE

scaler = StandardScaler()

rf_classifier = RandomForestClassifier()


# -------------------------------------------------
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rfe = RFE(estimator=rf_classifier, n_features_to_select=15)
X_train_rfe = rfe.fit_transform(X_train_scaled, y_train)
X_test_rfe = rfe.transform(X_test_scaled)

rf_classifier.fit(X_train_rfe, y_train)
y_pred = rf_classifier.predict(X_test_rfe)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')


# -------------------------------------------------
X_DPC_train_scaled = scaler.fit_transform(X_DPC_train)
X_DPC_test_scaled = scaler.transform(X_DPC_test)

rfe = RFE(estimator=rf_classifier, n_features_to_select=15)
X_DPC_train_rfe = rfe.fit_transform(X_DPC_train_scaled, y_DPC_train)
X_DPC_test_rfe = rfe.transform(X_DPC_test_scaled)

rf_classifier.fit(X_DPC_train_rfe, y_DPC_train)
y_DPC_pred = rf_classifier.predict(X_DPC_test_rfe)

accuracy = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy * 100:.2f}%')


# -------------------------------------------------
X_TPC_train_scaled = scaler.fit_transform(X_TPC_train)
X_TPC_test_scaled = scaler.transform(X_TPC_test)

rfe = RFE(estimator=rf_classifier, n_features_to_select=15)
X_TPC_train_rfe = rfe.fit_transform(X_TPC_train_scaled, y_TPC_train)
X_TPC_test_rfe = rfe.transform(X_TPC_test_scaled)

rf_classifier.fit(X_TPC_train_rfe, y_TPC_train)
y_TPC_pred = rf_classifier.predict(X_TPC_test_rfe)

accuracy = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy * 100:.2f}%')

Model Accuracy(AAC): 69.09%


KeyboardInterrupt: 

試試看針對字符出現的熵、偏度和峰度

In [51]:
from scipy.stats import skew, kurtosis
from collections import Counter

# 計算字符出現的概率
def calculate_probabilities(sequence):
    length = len(sequence)
    probabilities = {char: count / length for char, count in Counter(sequence).items()}
    return probabilities

# 計算熵
def calculate_entropy(sequence):
    probabilities = calculate_probabilities(sequence)
    entropy = -sum(p * (p and p != 1 and p != 0) * (p and p != 1 and p != 0) * (p and p != 1 and p != 0) for p in probabilities.values())
    return entropy

# 計算偏度
def calculate_skewness(sequence):
    frequencies = list(calculate_probabilities(sequence).values())
    skewness = skew(frequencies)
    return skewness

# 計算峰度
def calculate_kurtosis(sequence):
    frequencies = list(calculate_probabilities(sequence).values())
    kurt = kurtosis(frequencies)
    return kurt

# 對每個序列計算熵、偏度和峰度
X_train['Entropy'] = bench_train['Sequence'].apply(calculate_entropy)
X_train['Skewness'] = bench_train['Sequence'].apply(calculate_skewness)
X_train['Kurtosis'] = bench_train['Sequence'].apply(calculate_kurtosis)
X_test['Entropy'] = bench_test['Sequence'].apply(calculate_entropy)
X_test['Skewness'] = bench_test['Sequence'].apply(calculate_skewness)
X_test['Kurtosis'] = bench_test['Sequence'].apply(calculate_kurtosis)

# 對每個序列計算熵、偏度和峰度
X_DPC_train['Entropy'] = bench_train['Sequence'].apply(calculate_entropy)
X_DPC_train['Skewness'] = bench_train['Sequence'].apply(calculate_skewness)
X_DPC_train['Kurtosis'] = bench_train['Sequence'].apply(calculate_kurtosis)
X_DPC_test['Entropy'] = bench_test['Sequence'].apply(calculate_entropy)
X_DPC_test['Skewness'] = bench_test['Sequence'].apply(calculate_skewness)
X_DPC_test['Kurtosis'] = bench_test['Sequence'].apply(calculate_kurtosis)

# 對每個序列計算熵、偏度和峰度
X_TPC_train['Entropy'] = bench_train['Sequence'].apply(calculate_entropy)
X_TPC_train['Skewness'] = bench_train['Sequence'].apply(calculate_skewness)
X_TPC_train['Kurtosis'] = bench_train['Sequence'].apply(calculate_kurtosis)
X_TPC_test['Entropy'] = bench_test['Sequence'].apply(calculate_entropy)
X_TPC_test['Skewness'] = bench_test['Sequence'].apply(calculate_skewness)
X_TPC_test['Kurtosis'] = bench_test['Sequence'].apply(calculate_kurtosis)

# if X_train is NaN, set it to other's average
X_train['Entropy'] = X_train['Entropy'].fillna(X_train['Entropy'].mean())
X_train['Skewness'] = X_train['Skewness'].fillna(X_train['Skewness'].mean())
X_train['Kurtosis'] = X_train['Kurtosis'].fillna(X_train['Kurtosis'].mean())
X_test['Entropy'] = X_test['Entropy'].fillna(X_test['Entropy'].mean())
X_test['Skewness'] = X_test['Skewness'].fillna(X_test['Skewness'].mean())
X_test['Kurtosis'] = X_test['Kurtosis'].fillna(X_test['Kurtosis'].mean())
X_DPC_train['Entropy'] = X_DPC_train['Entropy'].fillna(X_DPC_train['Entropy'].mean())
X_DPC_train['Skewness'] = X_DPC_train['Skewness'].fillna(X_DPC_train['Skewness'].mean())
X_DPC_train['Kurtosis'] = X_DPC_train['Kurtosis'].fillna(X_DPC_train['Kurtosis'].mean())
X_DPC_test['Entropy'] = X_DPC_test['Entropy'].fillna(X_DPC_test['Entropy'].mean())
X_DPC_test['Skewness'] = X_DPC_test['Skewness'].fillna(X_DPC_test['Skewness'].mean())
X_DPC_test['Kurtosis'] = X_DPC_test['Kurtosis'].fillna(X_DPC_test['Kurtosis'].mean())
X_TPC_train['Entropy'] = X_TPC_train['Entropy'].fillna(X_TPC_train['Entropy'].mean())
X_TPC_train['Skewness'] = X_TPC_train['Skewness'].fillna(X_TPC_train['Skewness'].mean())
X_TPC_train['Kurtosis'] = X_TPC_train['Kurtosis'].fillna(X_TPC_train['Kurtosis'].mean())
X_TPC_test['Entropy'] = X_TPC_test['Entropy'].fillna(X_TPC_test['Entropy'].mean())
X_TPC_test['Skewness'] = X_TPC_test['Skewness'].fillna(X_TPC_test['Skewness'].mean())
X_TPC_test['Kurtosis'] = X_TPC_test['Kurtosis'].fillna(X_TPC_test['Kurtosis'].mean())

/var/folders/ks/8zyv9b656r5fyddgnypbjtpc0000gn/T/ipykernel_8437/427707448.py:19: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  skewness = skew(frequencies)
/var/folders/ks/8zyv9b656r5fyddgnypbjtpc0000gn/T/ipykernel_8437/427707448.py:25: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  kurt = kurtosis(frequencies)


In [84]:
from sklearn.preprocessing import StandardScaler

# -------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_scaled, y_train)

y_pred = rf_classifier.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')


# -------------------------------------------------
scaler = StandardScaler()
X_DPC_train_scaled = scaler.fit_transform(X_DPC_train)
X_DPC_test_scaled = scaler.transform(X_DPC_test)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_DPC_train_scaled, y_DPC_train)

y_DPC_pred = rf_classifier.predict(X_DPC_test_scaled)
accuracy = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy * 100:.2f}%')


# -------------------------------------------------
scaler = StandardScaler()
X_TPC_train_scaled = scaler.fit_transform(X_TPC_train)
X_TPC_test_scaled = scaler.transform(X_TPC_test)

rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_TPC_train_scaled, y_TPC_train)

y_TPC_pred = rf_classifier.predict(X_TPC_test_scaled)
accuracy = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy * 100:.2f}%')

Model Accuracy(AAC): 69.09%
Model Accuracy(DPC): 65.45%
Model Accuracy(TPC): 63.64%


處理 NT15 的資料

In [ ]:
with open('NT15dataset_train.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
nt15_train = pd.DataFrame(data)

print(nt15_train)
print()

amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

#--------------------------------------------------------------
# 計算每種胺基酸的出現次數
count_matrix_train = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_train[i, j] = sequence.count(amino_acid)
    percentage_matrix_train.append(count_matrix_train[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_train = np.array(percentage_matrix_train)

# 顯示比例矩陣
df_percentage_train = pd.DataFrame(percentage_matrix_train, columns=list(amino_acids))
print('AAC')
print(df_percentage_train)

# 合併特徵資料和目標資料
df_combined_train = pd.concat([df_percentage_train, nt15_train['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_train['Target'] = df_combined_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_train, y_train = df_combined_train.drop(['Header', 'Target'], axis=1), df_combined_train['Target']


#--------------------------------------------------------------
# 創建所有二個胺基酸相連的標籤
DPC = [a + b for a in amino_acids for b in amino_acids]

# 計算每對胺基酸的出現次數
count_DPC_train = np.zeros((len(sequences), len(DPC)))
percentage_DPC_train = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_train[i, j] = sequence.count(aa_pair)
    percentage_DPC_train.append(count_DPC_train[i]/15.)

# 計算每對胺基酸在序列中的比例
percentage_DPC_train = np.array(percentage_DPC_train)

# 顯示比例矩陣
df_percentage_DPC_train = pd.DataFrame(percentage_DPC_train, columns=DPC)
print('DPC:')
print(df_percentage_DPC_train)

# 合併特徵資料和目標資料
df_combined_DPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, nt15_train['Header']], axis=1)
print('AAC + DPC:')
print(df_combined_DPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_train['Target'] = df_combined_DPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_train, y_DPC_train = df_combined_DPC_train.drop(['Header', 'Target'], axis=1), df_combined_DPC_train['Target']
#print(X_DPC_train)

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_train = np.zeros((len(sequences), len(TPC)))
percentage_TPC_train = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_train[i, j] = sequence.count(amino_acid_combination)
        
    percentage_TPC_train.append(count_TPC_train[i]/15.)

# 計算每種胺基酸在序列中的比例
percentage_TPC_train = np.array(percentage_TPC_train)

# 顯示比例矩陣
df_percentage_TPC_train = pd.DataFrame(percentage_TPC_train, columns=TPC)
print('TPC:')
print(df_percentage_TPC_train)

# 合併特徵資料和目標資料
df_combined_TPC_train = pd.concat([df_percentage_train, df_percentage_DPC_train, df_percentage_TPC_train, nt15_train['Header']], axis=1)
print('AAC + DPC + TPC:')
print(df_combined_TPC_train)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_train['Target'] = df_combined_TPC_train['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_train, y_TPC_train = df_combined_TPC_train.drop(['Header', 'Target'], axis=1), df_combined_TPC_train['Target']

In [ ]:
with open('NT15dataset_test.fasta', 'r') as file:
    lines = file.readlines()

# 初始化兩個空的列表，分別用於存儲標題（Header）和序列（Sequence）
headers = []
sequences = []

# 遍歷 FASTA 文件的每一行
for line in lines:
    line = line.strip()  # 去掉行尾的空白字符

    # 如果行以 '>AA' 開頭，則視為標題
    if line.startswith('>AA') or line.startswith('>neg'):
        headers.append(line)
        sequences.append('')
    else:
        sequences[-1] += line

# 將標題和序列轉換為 DataFrame
data = {'Header': headers, 'Sequence': sequences}
nt15_test = pd.DataFrame(data)

amino_acids = 'ACDEFGHIKLMNPQRSTVWY'


#--------------------------------------------------------------
# 計算每種胺基酸的出現次數
count_matrix_test = np.zeros((len(sequences), len(amino_acids)))
percentage_matrix_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid in enumerate(amino_acids):
        count_matrix_test[i, j] = sequence.count(amino_acid)
    
    percentage_matrix_test.append(count_matrix_test[i]/len(sequence))

# 計算每種胺基酸在序列中的比例
percentage_matrix_test = np.array(percentage_matrix_test)

# 顯示比例矩陣
df_percentage_test = pd.DataFrame(percentage_matrix_test, columns=list(amino_acids))
print('AAC:')
print(df_percentage_test)

# 合併特徵資料和目標資料
df_combined_test = pd.concat([df_percentage_test, nt15_test['Header']], axis=1)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_test['Target'] = df_combined_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_test, y_test = df_combined_test.drop(['Header', 'Target'], axis=1), df_combined_test['Target']

#print(X_test)
#print(y_test)

#--------------------------------------------------------------
# 創建所有二個胺基酸相連的標籤
DPC = [a + b for a in amino_acids for b in amino_acids]

# 計算每對胺基酸的出現次數
count_DPC_test = np.zeros((len(sequences), len(DPC)))
percentage_DPC_test = []

for i, sequence in enumerate(sequences):
    for j, aa_pair in enumerate(DPC):
        count_DPC_test[i, j] = sequence.count(aa_pair)
        
    percentage_DPC_test.append(count_DPC_test[i]/15.)

# 計算每對胺基酸在序列中的比例
percentage_DPC_test = np.array(percentage_DPC_test)

# 顯示比例矩陣
df_percentage_DPC_test = pd.DataFrame(percentage_DPC_test, columns=DPC)
print('DPC:')
print(df_percentage_DPC_test)

# 合併特徵資料和目標資料
df_combined_DPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, nt15_test['Header']], axis=1)
print('AAC + DPC:')
print(df_combined_DPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_DPC_test['Target'] = df_combined_DPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_DPC_test, y_DPC_test = df_combined_DPC_test.drop(['Header', 'Target'], axis=1), df_combined_DPC_test['Target']
#print(X_DPC_train)

#--------------------------------------------------------------
# 產生所有三種胺基酸的排列組合
TPC = [''.join(comb) for comb in product(amino_acids, repeat=3)]

# 計算每種胺基酸的出現次數
count_TPC_test = np.zeros((len(sequences), len(TPC)))
percentage_TPC_test = []

for i, sequence in enumerate(sequences):
    for j, amino_acid_combination in enumerate(TPC):
        count_TPC_test[i, j] = sequence.count(amino_acid_combination)
        
    percentage_TPC_test.append(count_TPC_test[i]/15.)

# 計算每種胺基酸在序列中的比例
percentage_TPC_test = np.array(percentage_TPC_test)

# 顯示比例矩陣
df_percentage_TPC_test = pd.DataFrame(percentage_TPC_test, columns=TPC)
print('TPC:')
print(df_percentage_TPC_test)

# 合併特徵資料和目標資料
df_combined_TPC_test = pd.concat([df_percentage_test, df_percentage_DPC_test, df_percentage_TPC_test, nt15_test['Header']], axis=1)
print('AAC + DPC + TPC:')
print(df_combined_TPC_test)

# 將目標轉換為二元標籤（1 表示 'AA'，0 表示 'neg'）
df_combined_TPC_test['Target'] = df_combined_TPC_test['Header'].apply(lambda x: 1 if 'AA' in x else 0)

# 分割資料集
X_TPC_test, y_TPC_test = df_combined_TPC_test.drop(['Header', 'Target'], axis=1), df_combined_TPC_test['Target']

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier()

# single
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy(AAC): {accuracy * 100:.2f}%')

# DPC
rf_classifier.fit(X_DPC_train, y_DPC_train)
y_DPC_pred = rf_classifier.predict(X_DPC_test)

accuracy_DPC = accuracy_score(y_DPC_test, y_DPC_pred)
print(f'Model Accuracy(DPC): {accuracy_DPC * 100:.2f}%')

# TPC
rf_classifier.fit(X_TPC_train, y_TPC_train)
y_TPC_pred = rf_classifier.predict(X_TPC_test)

accuracy_TPC = accuracy_score(y_TPC_test, y_TPC_pred)
print(f'Model Accuracy(TPC): {accuracy_TPC * 100:.2f}%')